# Calculating the Discrete Fréchet Distance
The discrete Fréchet distance measures the similarity between two polygonal curves or poly-lines.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math

from distances.discrete import Frechet

%matplotlib inline

In [5]:
np.set_printoptions(precision=4)

In [17]:
def bresenham_pairs(x0, y0, x1, y1):
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    dim = max(dx, dy)
    i = 0
    pairs = np.zeros((dim, 2), dtype=int)
    x, y = x0, y0
    sx = -1 if x0 > x1 else 1
    sy = -1 if y0 > y1 else 1
    if dx > dy:
        err = dx // 2
        while x != x1:
            #self.set(x, y)
            pairs[i] = np.array([x, y])
            i += 1
            err -= dy
            if err < 0:
                y += sy
                err += dx
            x += sx
    else:
        err = dy // 2
        while y != y1:
            #self.set(x, y)
            pairs[i] = np.array([x, y])
            i += 1
            err -= dx
            if err < 0:
                x += sx
                err += dy
            y += sy        
    #self.set(x, y)
    # pairs[i] = np.array([x, y])
    return pairs

In [18]:
p = bresenham_pairs(0, 0, 12, 7)

In [19]:
p

array([[ 0,  0],
       [ 1,  1],
       [ 2,  1],
       [ 3,  2],
       [ 4,  2],
       [ 5,  3],
       [ 6,  3],
       [ 7,  4],
       [ 8,  5],
       [ 9,  5],
       [10,  6],
       [11,  6]])

The `euclidean` function calculates the distance between two points in *R*<sup>N</sup>

In [47]:
def euclidean_pairwise(p: np.ndarray, q: np.ndarray) -> np.ndarray:
    d = p - q
    return np.sqrt(d[:,0] ** 2 + d[:,1] ** 2)

In [4]:
def np_euclidean(p: np.ndarray, q: np.ndarray) -> np.ndarray:
    """
    Calculates the point-to-point distance between poly-lines p and q
    :param p: Poly-line p
    :param q: Poly-line q
    :return: Distance array
    """
    n_p = p.shape[0]
    n_q = q.shape[0]
    pp = np.repeat(p, n_q, axis=0)
    qq = np.tile(q, (n_p, 1))
    dd = pp - qq
    dist = np.sqrt(dd[:, 0] ** 2 + dd[:, 1] ** 2).reshape(n_p, n_q)
    return dist

In [22]:
p = np.array([[0.0, 0.0], [1.0, 0.0], [2.0, 0.0], [3.0, 0.0], [3.0, -1.0], [2.0, -1.0], [2.0, -2.0]])

In [23]:
q = np.array([[0.0, 1.0], [1.0, 1.1], [2.5, 1.3],  [2.0, 1.2], [3.0, 1.1], [4.0, 1.0]])

In [7]:
frechet = Frechet(np_euclidean)

In [9]:
#%%timeit
frechet.distance(p, q)

3.605551275463989

In [ ]:
fsd = frechet.ca

In [ ]:
pp = plt.imshow(fsd)

In [ ]:
fsd

In [ ]:
b = (fsd <= 3.6)

In [ ]:
pp = plt.imshow(b)

In [ ]:
dist = np.zeros((p.shape[0], q.shape[0]))

In [ ]:
fsd

In [24]:
#%%timeit
n_p = p.shape[0]
n_q = q.shape[0]
pp = np.repeat(p, n_q, axis=0)
qq = np.tile(q, (n_p, 1))
dd = pp - qq
dist = np.sqrt(dd[:, 0] ** 2 + dd[:, 1] ** 2).reshape(n_p, n_q)
fd = np.max(np.concatenate((dist.min(axis=1), (dist.min(axis=0)))))

In [28]:
bp = bresenham_pairs(0, 0, n_p, n_q)

In [29]:
bp

array([[0, 0],
       [1, 1],
       [2, 2],
       [3, 3],
       [4, 3],
       [5, 4],
       [6, 5]])

In [33]:
p[bp[:,0]]

array([[ 0.,  0.],
       [ 1.,  0.],
       [ 2.,  0.],
       [ 3.,  0.],
       [ 3., -1.],
       [ 2., -1.],
       [ 2., -2.]])

In [35]:
q[bp[:,1]]

array([[0. , 1. ],
       [1. , 1.1],
       [2.5, 1.3],
       [2. , 1.2],
       [2. , 1.2],
       [3. , 1.1],
       [4. , 1. ]])

In [49]:
ep = euclidean_pairwise(p[bp[:,0]], q[bp[:,1]])

In [50]:
ep

array([1.    , 1.1   , 1.3928, 1.562 , 2.4166, 2.3259, 3.6056])